In [1]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [2]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt

d:\CODING\PYTHON\MACHINE_LEARNING\Tenangin\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [3]:
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from langchain.embeddings.base import Embeddings
import torch
from langchain.schema import Document

# Load SBERT model dan pastikan menggunakan GPU jika tersedia
device = 'cuda' if torch.cuda.is_available() else 'cpu'
sbert_model = model = SentenceTransformer('naufalihsan/indonesian-sbert-large')
sbert_model = sbert_model.to(device)  # Pindahkan model ke GPU (jika ada)

# Custom embeddings class for SBERT
class SBERTEmbeddings(Embeddings):
    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        # Menggunakan model SBERT untuk menghasilkan embeddings
        embeddings = sbert_model.encode(texts, convert_to_tensor=True, show_progress_bar=True)
        embeddings = embeddings.to(device)  # Pindahkan embeddings ke GPU (jika ada)
        return embeddings.cpu().numpy().tolist()  # Pindahkan kembali ke CPU untuk konversi

    def embed_query(self, query: str) -> list[float]:
        # Menghasilkan embedding untuk query
        embedding = sbert_model.encode(query, convert_to_tensor=True)
        embedding = embedding.to(device)  # Pindahkan embedding ke GPU (jika ada)
        return embedding.cpu().numpy().tolist()  # Pindahkan kembali ke CPU untuk konversi

# Inisialisasi embeddings SBERT dan FAISS vector store
sbert_embeddings = SBERTEmbeddings()

d:\CODING\PYTHON\MACHINE_LEARNING\Tenangin\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\CODING\PYTHON\MACHINE_LEARNING\Tenangin\.venv\Lib\site-packages\transformers\utils\hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
vector_store = FAISS.load_local("tudocs_faiss",sbert_embeddings,allow_dangerous_deserialization=True)


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [10]:
response = graph.invoke({"question": "kenapa ya, saya merasa tidak berguna. saya kurang dihargai orang lain karena hal itu."})
print(f"Anda    : {response['question']}")
print(f"Gemini  : {response['answer']}")
print(f"Context : {response['context'][0].page_content}")

Anda    : kenapa ya, saya merasa tidak berguna. saya kurang dihargai orang lain karena hal itu.
Gemini  : Merasa tidak berguna dan kurang dihargai bisa disebabkan oleh beberapa faktor, seperti gangguan kepribadian menghindar yang membuat individu takut dihakimi dan memiliki harga diri rendah. Kurangnya motivasi juga bisa menjadi penyebab, yang mana setiap orang pasti mengalaminya. Interaksi yang penuh kasih dan tidak menghakimi dari orang tua sangat penting untuk mengatasi motivasi yang rendah.
Context : orang lain mengambil tanggung jawab atas sebagian hidupnya. Seseorang merasa kurang percaya diri atau tidak mampu melakukan hal-hal normal sendirian sehingga menempatkan kebutuhan diri sendiri di atas kebutuhan orang lain, dan merasa putus asa atau takut sendirian. 2) Gangguan kepribadian menghindar Jika memiliki gangguan kepribadian menghindar, mungkin takut dihakimi secara negatif. Ini dapat menyebabkan induvidu merasa tidak nyaman dalam situasi sosial. Seserang akan merasa peka terh